In [3]:
import pandas as pd
import pystac_client
import planetary_computer

from eodal.core.band import Band
from eodal.core.scene import SceneCollection
from eodal.core.raster import RasterCollection, SceneProperties

# define a bounding box
bbox = [9.0924, 47.5992, 9.2190, 47.7295]
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
# search the catalog by bounding box and a custom time period
search = catalog.search(
    collections=["landsat-c2-l2"], bbox=bbox, datetime=["2021-07-01", "2021-07-31"]
)
# query the catalog
items = search.get_all_items()
print(f"Found {len(items)} items")

# read the data into EOdal -> we will get a SceneCollection
sc = SceneCollection()

# loop over items (i.e., single Landsat scenes)
for item in items:
    # get some metadata
    scene_props = SceneProperties(
        product_uri=item.id,
        acquisition_time=pd.to_datetime(item.properties['datetime']),
        platform=item.properties['platform'],
        sensor=item.properties['instruments'][0])
    
    rc = RasterCollection()
    for band_name in ['blue', 'red', 'green', 'nir08', 'swir16', 'swir22', 'qa_pixel']:
        band = Band.from_rasterio(
            planetary_computer.sign_url(item.assets[band_name].href),
            band_name_dst=band_name
        )
        rc.add_band(band)
    # add the scene properties to RasterCollection
    rc.scene_properties = scene_props
    # add the scene to the SceneCollection
    sc.add_scene(rc)

# the data is now stored in sc -> you can work with as with any other SceneCollection object
sc


Found 8 items


EOdal SceneCollection
----------------------
# Scenes:    8
Timestamps:    2021-07-30 09:11:53.477169+00:00, 2021-07-29 10:16:32.656734+00:00, 2021-07-22 10:10:18.192019+00:00, 2021-07-21 09:18:51.423700+00:00, 2021-07-14 09:13:16.812454+00:00, 2021-07-13 10:16:25.078262+00:00, 2021-07-06 10:10:13.986245+00:00, 2021-07-05 09:20:13.621974+00:00
Scene Identifiers:    LE07_L2SP_194027_20210730_02_T1, LC08_L2SP_195027_20210729_02_T1, LC08_L2SP_194027_20210722_02_T1, LE07_L2SP_195027_20210721_02_T1, LE07_L2SP_194027_20210714_02_T2, LC08_L2SP_195027_20210713_02_T2, LC08_L2SP_194027_20210706_02_T1, LE07_L2SP_195027_20210705_02_T2